In [1]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy # Plotting tools
import nltk
import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim


In [2]:
# Downloading stopwords from nltk package
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

c:\Python3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Lucas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
# Loading Dataset
df=pd.read_json('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
print(df.target_names.unique())
df.head()

# Removing redundant data from dataframe
data = df.content.values.tolist()  
# Remove Emails 
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]  
# Remove new line characters 
data = [re.sub('\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]  
pprint(data[:1])

print(len(data))

c:\Python3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
<>:9: DeprecationWarning: invalid escape sequence '\S'
<>:11: DeprecationWarning: invalid escape sequence '\s'
<>:9: DeprecationWarning: invalid escape sequence '\S'
<>:11: DeprecationWarning: invalid escape sequence '\s'
<ipython-input-3-9e3d14bfbe41>:9: DeprecationWarning: invalid escape sequence '\S'
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-3-9e3d14bfbe41>:11: DeprecationWarning: invalid escape sequence '\s'
  data = [re.sub('\s+', ' ', sent) for sent in data]


['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']
['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 't

In [4]:
# Tokenizing words
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

# Creating bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])


c:\Python3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]
['from', 'wheres', 'my', 'thi

In [5]:
# Function to remove stopwords
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

# Functions to create bigrams and trigrams
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

# Functions to lemmatize words
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

c:\Python3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [6]:
# Running the functions above
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

c:\Python3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[['s', 'thing', 'car', 'nntp_poste', 'host', 'rac_wam', 'university', 'park', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


In [7]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts] 

# View 
print(corpus[:1])   # produces indexes for word id and word freq
#[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]] #readable form of corpus

c:\Python3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1)]]


In [9]:
import time

start = time.time()
# Building topic model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

end = time.time()

time_taken = end - start
print(time_taken)



c:\Python3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


98.14753699302673


In [10]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]


[(0,
  '0.021*"research" + 0.019*"information" + 0.019*"high" + 0.019*"report" + '
  '0.018*"player" + 0.016*"service" + 0.015*"rate" + 0.014*"design" + '
  '0.013*"season" + 0.012*"low"'),
 (1,
  '0.077*"team" + 0.072*"game" + 0.053*"play" + 0.050*"faith" + 0.049*"win" + '
  '0.031*"belief" + 0.025*"atheist" + 0.025*"year" + 0.018*"wing" + '
  '0.018*"score"'),
 (2,
  '0.106*"space" + 0.029*"notice" + 0.029*"launch" + 0.026*"earth" + '
  '0.024*"mission" + 0.024*"orbit" + 0.023*"external" + 0.020*"vehicle" + '
  '0.019*"satellite" + 0.019*"door"'),
 (3,
  '0.022*"say" + 0.019*"people" + 0.017*"reason" + 0.017*"believe" + '
  '0.015*"evidence" + 0.014*"mean" + 0.012*"point" + 0.012*"question" + '
  '0.011*"many" + 0.010*"claim"'),
 (4,
  '0.078*"book" + 0.044*"science" + 0.042*"reference" + 0.036*"pin" + '
  '0.032*"section" + 0.025*"faq" + 0.024*"author" + 0.023*"copy" + '
  '0.023*"reality" + 0.022*"internal"'),
 (5,
  '0.065*"cost" + 0.059*"model" + 0.039*"character" + 0.036*"pictur

c:\Python3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [32]:

lda_model.save("ldaModel_english.model")

In [15]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# # Compute Coherence Score - C_v 
# coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_lda = coherence_model_lda.get_coherence()
# print('\nCoherence Score: ', coherence_lda)

# U_mass
coherence_model_lda = CoherenceModel(model=lda_model, corpus=corpus, dictionary=id2word, coherence='u_mass')
u_mass = coherence_model_lda.get_coherence()
print('\nU_mass: ', coherence_lda)

c:\Python3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



Perplexity:  -13.32461012565659

Coherence Score:  0.483541481988623

U_mass:  0.483541481988623


In [12]:
pyLDAvis.enable_notebook()

visE = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
visE

c:\Python3\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
c:\Python3\lib\site-packages\pyLDAvis\_prepare.py:244: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
18     0.336524 -0.053715       1        1  34.413837
3      0.316929 -0.118906       2        1  15.585653
11     0.216786  0.285241       3        1   9.181642
6      0.247092  0.167002       4        1   9.110785
13     0.184109 -0.279644       5        1   7.614212
10    -0.019302  0.039641       6        1   5.391939
0      0.153914  0.027981       7        1   5.120596
9     -0.008546 -0.117700       8        1   2.140505
1      0.006923 -0.062094       9        1   2.065612
19     0.038372  0.069508      10        1   1.847951
16    -0.037562  0.037040      11        1   1.732311
4     -0.046036 -0.038088      12        1   1.606862
2     -0.081906  0.028035      13        1   1.237493
5     -0.118579  0.023843      14        1   1.144280
8     -0.162683  0.013118      15        1   0.619225
17    -0.175944 -0.007820      16        1   0.433673
15    -0.204259 -0.001778      17        1   0.294853
7     -0.212202 -0.003263      18        1   0.200450
14    -0.214082 -0.003632      19        1   0.158955
12    -0.219547 -0.004767      20        1   0.099166, topic_info=              Term          Freq         Total Category  logprob  loglift
4399            ax  53235.000000  53235.000000  Default  30.0000  30.0000
17            line  17060.000000  17060.000000  Default  29.0000  29.0000
208          write  13405.000000  13405.000000  Default  28.0000  28.0000
142   organization   9429.000000   9429.000000  Default  27.0000  27.0000
45         article   9896.000000   9896.000000  Default  26.0000  26.0000
...            ...           ...           ...      ...      ...      ...
4946            tv      0.020348      1.258241  Topic20 -11.0492   2.7917
6485   advertising      0.020348      1.258227  Topic20 -11.0492   2.7917
7775          logo      0.020348      1.258225  Topic20 -11.0492   2.7917
7567        circle      0.020348      1.258226  Topic20 -11.0492   2.7917
4391             _      0.020348      1.258251  Topic20 -11.0492   2.7916

[871 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
495       9  0.990580    absolute
1354      2  0.999138      accept
1277     15  0.843120  accessible
2283      6  0.995016  accomplish
183       1  0.170685     address
...     ...       ...         ...
40        8  0.002984        year
40        9  0.134080        year
614       1  0.590722       young
614       8  0.374635       young
614       9  0.031965       young

[1018 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[19, 4, 12, 7, 14, 11, 1, 10, 2, 20, 17, 5, 3, 6, 9, 18, 16, 8, 15, 13])